## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import requests
import pandas as pd
import json
from datetime import datetime

## Download

Retrieve the page

In [6]:
url = "https://services1.arcgis.com/UvqJJ6GFv4u5BZQj/arcgis/rest/services/CoronavirusCases_current_85845257045d47489899f0cb72b91eee/FeatureServer/0//query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [7]:
r = requests.get(url)

In [8]:
data = r.json()

## Parse

In [9]:
dict_list = []

In [10]:
for item in data["features"]:
    d = dict(
        county="Nevada",
        zip_code=item["attributes"]["name"],
        confirmed_cases=item["attributes"]["positive"],
    )
    dict_list.append(d)

In [11]:
df = pd.DataFrame(dict_list)

De-id

In [12]:
df["confirmed_cases"] = df["confirmed_cases"].astype(int)

In [13]:
df.loc[df["confirmed_cases"] < 10, "confirmed_cases"] = "<10"

Drop unneeded rows

In [14]:
todrop = ["Nevada", "West", "East"]

In [15]:
df = df[~df["zip_code"].isin(todrop)]

Match city names to zip codes

In [16]:
zip_codes = [
    "95712",
    "95924",
    "96111",
    "95724",
    "95728",
    "95960",
    "95602",
    "95977",
    "95945",
    "95949",
    "95975",
    "95946",
    "95959",
    "95986",
    "96162",
    "96161",
    "96160",
]

In [17]:
cities = [
    "Chicago Park",
    "Cedar Ridge",
    "Floriston",
    "Norden",
    "Soda Springs",
    "North San Juan",
    "Auburn",
    "Smartville",
    "Grass Valley",
    "Grass Valley",
    "Rough And Ready",
    "Penn Valley",
    "Nevada City",
    "Washington",
    "Truckee",
    "Truckee",
    "Truckee",
]

In [18]:
d = {"zip_code": zip_codes, "city": cities}

In [19]:
matchup_df = pd.DataFrame(d)

In [20]:
merge_df = pd.merge(df, matchup_df, how="left", on="zip_code")

In [21]:
merge_df["area"] = merge_df["zip_code"] + ": " + merge_df["city"]

Get timestamp

In [22]:
date_url = "https://services1.arcgis.com/UvqJJ6GFv4u5BZQj/arcgis/rest/services/CoronavirusCases_current_85845257045d47489899f0cb72b91eee/FeatureServer/0?f=json"
date_r = requests.get(date_url)
date_data = date_r.json()

In [23]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [24]:
timestamp = datetime.fromtimestamp((timestamp / 1000))

In [25]:
latest_date = pd.to_datetime(timestamp).date()

In [26]:
merge_df["county_date"] = latest_date

Clean up for export

In [27]:
export_df = merge_df[
    ["county", "area", "confirmed_cases", "county_date", "zip_code"]
].rename(columns={"zip_code": "zip"})

## Vet

In [28]:
try:
    assert not len(export_df) < 17
except AssertionError:
    raise AssertionError("Nevada County's scraper is missing rows")

In [29]:
try:
    assert not len(export_df) > 17
except AssertionError:
    raise AssertionError("Nevada County's scraper has more rows than before")

## Export

Set date

In [30]:
tz = pytz.timezone("America/Los_Angeles")

In [31]:
today = datetime.now(tz).date()

In [32]:
slug = "nevada"

In [33]:
export_df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [34]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [35]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [36]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [37]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)